In [2]:
import os
import time
from osgeo import gdal
from utilis import *
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# 定義植生指標
VIs = ['NDVI', 'NDRE', 'NDI', 'RGI', 'ExG']

# 定義主要輸入文件夾
main_input_folder = '04_Result/Image/UC/ROI'
output_base_folder = '04_Result/Image/UC/ROI'

# 記錄總開始時間
total_start_time = time.time()

# 遍歷 ROI 資料夾中的所有日期資料夾
for i, date in enumerate(os.listdir(main_input_folder), 1):
    input_folder = os.path.join(main_input_folder, date, 'VI')
    output_folder_independent = os.path.join(output_base_folder, date, 'VI', 'Independent')
    source_folder = os.path.join(main_input_folder, date, 'tif')

    print('------------------------------')
    print(f"開始處理第 {i} 個資料夾，日期為:{date}")
    dir_maker(output_folder_independent)
    iter_start_time = time.time()

    for VI in VIs:
        for vi_file in os.listdir(input_folder):
            if VI in vi_file and vi_file.endswith('.tif'):
                input_vi_file = os.path.join(input_folder, vi_file)
                
                # 根據 VI 去掉前綴找到對應的 source_file
                base_filename = vi_file.replace(f'{VI}_', '')
                source_file = os.path.join(source_folder, base_filename)

                # 檢查 source_file 是否存在
                if not os.path.exists(source_file):
                    print(f"Source file {source_file} does not exist, skipping.")
                    continue

                # 讀取 TIFF 檔案
                dataset = gdal.Open(input_vi_file)
                band = dataset.GetRasterBand(1)
                vi_data = band.ReadAsArray()

                # 讀取 source_file 檔案作為 mask
                dataset_source = gdal.Open(source_file)
                if dataset_source is None:
                    print(f"Failed to open {source_file}, skipping.")
                    continue
                
                try:
                    mask_band = dataset_source.GetRasterBand(7)
                except RuntimeError as e:
                    print(f"Failed to get raster band: {e}, skipping.")
                    continue

                mask_data = mask_band.ReadAsArray()
                
                # 將 mask_data 的值縮放到 0 到 1 之間
                mask_data = mask_data / 65535.0

                # 將 mask_data 中的 NaN 填充為 0
                mask_data = np.nan_to_num(mask_data, nan=0)

                # 處理 vi_data 和 mask_data
                mask = mask_data == 1  # 只選擇 mask_data 等於 1 的部分

                # 把不符合 mask 的區域設為 NaN
                vi_data_masked = np.where(mask, vi_data, np.nan)

                # 計算 95% 百分位數
                percentile_95 = np.nanpercentile(vi_data_masked, 95)

                # 將大於 95% 百分位數的值替換為 95% 百分位數的值
                vi_data_masked[vi_data_masked > percentile_95] = percentile_95

                # 創建顏色圖譜，將 NaN 值設置為透明
                cmap = plt.get_cmap('RdYlGn')
                cmap = cmap(np.arange(cmap.N))
                cmap[:, -1] = np.where(np.isnan(cmap[:, -1]), 0, 1)  # 設置透明度
                transparent_cmap = mcolors.ListedColormap(cmap)

                # 顯示圖像並保存成 PNG 檔案（個別檔案）
                individual_start_time = time.time()
                fig, ax = plt.subplots()
                im = ax.imshow(vi_data_masked, cmap=transparent_cmap, vmin=0, vmax=1)
                ax.axis('off')  # 關閉X、Y軸
                ax.set_frame_on(False)  # 去掉框框
                output_png_file_independent = os.path.join(output_folder_independent, vi_file.replace('.tif', '.png'))
                plt.savefig(output_png_file_independent, bbox_inches='tight', pad_inches=0, transparent=True)
                plt.close()
                individual_end_time = time.time()
                print(f'{date}_{VI}_{base_filename} 檔案處理花費時間: {individual_end_time - individual_start_time:.2f} 秒')
    
    iter_end_time = time.time()
    print(f"第 {i} 個資料夾處理結束，花費時間: {iter_end_time - iter_start_time:.2f} 秒")
    print('------------------------------')
    print()

# 計算並顯示總時間
total_end_time = time.time()
print(f'處理完成，共花費時間: {total_end_time - total_start_time:.2f} 秒')


------------------------------
開始處理第 1 個資料夾，日期為:20240415
20240415_NDVI_乾旱處理_(DD_1)_0.tif 檔案處理花費時間: 2.11 秒
20240415_NDVI_乾旱處理_(DD_2)_0.tif 檔案處理花費時間: 1.38 秒
20240415_NDVI_對照組_(CK)_0.tif 檔案處理花費時間: 2.91 秒
20240415_NDRE_乾旱處理_(DD_1)_0.tif 檔案處理花費時間: 2.34 秒
20240415_NDRE_乾旱處理_(DD_2)_0.tif 檔案處理花費時間: 1.35 秒
20240415_NDRE_對照組_(CK)_0.tif 檔案處理花費時間: 2.88 秒
20240415_NDI_乾旱處理_(DD_1)_0.tif 檔案處理花費時間: 2.22 秒
20240415_NDI_乾旱處理_(DD_2)_0.tif 檔案處理花費時間: 1.58 秒
20240415_NDI_對照組_(CK)_0.tif 檔案處理花費時間: 2.77 秒
20240415_RGI_乾旱處理_(DD_1)_0.tif 檔案處理花費時間: 2.36 秒
20240415_RGI_乾旱處理_(DD_2)_0.tif 檔案處理花費時間: 1.30 秒
20240415_RGI_對照組_(CK)_0.tif 檔案處理花費時間: 3.03 秒
20240415_ExG_乾旱處理_(DD_1)_0.tif 檔案處理花費時間: 1.98 秒
20240415_ExG_乾旱處理_(DD_2)_0.tif 檔案處理花費時間: 1.30 秒
20240415_ExG_對照組_(CK)_0.tif 檔案處理花費時間: 2.78 秒
處理完成，共花費時間: 72.42 秒
